In [100]:
import requests
from lxml import html

In [118]:
rss="https://feeds.elpais.com/mrss-s/pages/ep/site/elpais.com/portada"
urls=["https://elpais.com/","https://elpais.com/tecnologia/"]

In [126]:
def scapArticuloPais(idx, noticias):
    print(idx)
    for i,urlNoticia in enumerate(noticias):
        #print(items[i].text)
        noticia=html.fromstring(requests.get(urlNoticia).content)
        #print(noticia)
        tituloNoticia=noticia.xpath("//h1[@id='articulo-titulo']")
        fichero="texto "+str(idx)+"-"+str(i)+".txt"
        titulo=""
        f = open("textos\\"+fichero, "w")
        if(len(tituloNoticia)>0):
            #print(tituloNoticia[0].text)
            titulo=tituloNoticia[0].text
            #fichero=titulo+".txt"
        f.write(titulo.encode()+"\n")
        entradillaNoticia=noticia.xpath("//h2[@class='articulo-subtitulo']")
        if(len(entradillaNoticia)>0):
            #print(entradillaNoticia[0].text)
            f.write(entradillaNoticia[0].text.encode()+"\n")
        cuerposNoticia=noticia.xpath("//div[@id='cuerpo_noticia']/p")
        for i,cuerpo in enumerate(cuerposNoticia):
            if i>0:
                f.write("\n")
            for contenido in cuerpo.itertext():
                #print(cuerpo)
                f.write(contenido.encode())
            
        #print("--------------------")
        f.close()
    

In [128]:
#noticias=html.fromstring(requests.get(rss).content)
#items=noticias.xpath("//item/guid")
idx=0
for url in urls:
    pagina=html.fromstring(requests.get(url).content)
    articulos=pagina.xpath("//article")
    noticias=[]
    for articulo in articulos:
        enlace=articulo.findall("h2/a")
        if(len(enlace)>0):
            href=enlace[0].attrib.get("href")
            if(href[0]=="/"):
                #print(href)
                noticias.append(url+href)
    scapArticuloPais(idx, noticias)
    idx=idx+1
##print(noticias)

0


TypeError: can't concat str to bytes

In [99]:
url="https://projects.fivethirtyeight.com/soccer-predictions/matches/"

pagina=html.fromstring(requests.get(url).content)
tbody=pagina.xpath("//table[@id='all-matches']/tbody")
for fila in tbody[0].getchildren():
    celdas=fila.findall("td")
    print(len(celdas))
    print(celdas[0].findall("div[@class='time-league']")[0].text)
    #for texto in celdas[0].itertext():
    #    print(texto)



6
Bundesliga
6
First Division A
6
Championship
6
Ligue 1
6
Serie A
6
La Liga
6
La Liga 2
6
Serie B
6
Primeira Liga
6
Superliga
6
Superliga
6
Liga MX
6
Liga MX
6
Liga MX
6
A-League
6
A-League
6
Süper Lig
6
2. Bundesliga
6
2. Bundesliga
6
2. Bundesliga
6
2. Bundesliga
6
La Liga
6
Championship
6
Premier League
6
Ligue 2
6
Serie A
6
Serie B
6
Serie B
6
Serie B
6
Serie B
6
Süper Lig
6
Bundesliga
6
Bundesliga
6
Bundesliga
6
Bundesliga
6
Championship
6
Championship
6
Championship
6
Championship
6
Championship
6
Championship
6
Championship
6
Championship
6
Championship
6
Championship
6
La Liga
6
La Liga 2
6
La Liga 2
6
League One
6
League One
6
League One
6
League One
6
League One
6
League One
6
League One
6
League One
6
League One
6
League One
6
League One
6
League Two
6
League Two
6
League Two
6
League Two
6
League Two
6
League Two
6
League Two
6
League Two
6
League Two
6
League Two
6
League Two
6
League Two
6
Premier League
6
Premier League
6
Premier League
6
Premier League
6
Premiership
6
